<a href="https://colab.research.google.com/github/nullzero-live/productAgent/blob/main/market_research_CAMEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this tutorial, we'll walk you through the development of an end-to-end Machine Learning software project that leverages an exciting and cutting-edge concept: CAMEL LangChain. We'll guide you from product description to deployment on Google Cloud Platform, using a wide range of software tools to ensure a comprehensive understanding.

Phase 1: Experimenting with CAMEL LangChain
This phase of the project has the lowest probability of success but is the most exciting conceptually. The idea published by the authors is philosophically and technologically interesting.

CAMEL LangChain is an implementation found on Github that employs a unique approach called "inception prompting." This approach encourages a pair of agents to be guided toward a goal. In this experimental phase, we'll treat these agents like dueling musicians, each with a common goal. The authors of the paper attempt to understand the "cognition" of the Large Language Models (LLMs) these agents are part of.

To learn more about CAMEL LangChain, you can visit the project website or read the Arxiv paper.

Steps in this phase
Create a pair of agents discussing the startup's market: Our goal is to have these agents analyze not only the options of the market but also its depth, if possible in the implementation.

Compress the discussion: Extract keywords from the agent's conversation and organize them into a data structure.

Embed the discussion using OpenAI Ada embeddings: Transform the compressed output into a suitable format for storage.

Finally, we'll store the compressed output in a vector store on Pinecone.

Source https://python.langchain.com/en/latest/use_cases/agent_simulations/camel_role_playing.html

In [ ]:
''' A pair of agents discussing the startup idea and storing the output
in a vectorstore on pinecone '''

import os
from dotenv import load_dotenv

from typing import List
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)


In [ ]:

# Setup a CAMEL agent helper class to manage 

class CAMELAgent:

    def __init__(
        self,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.system_message = system_message
        self.model = model
        self.init_messages()

    def reset(self) -> None:
        self.init_messages()
        return self.stored_messages

    def init_messages(self) -> None:
        self.stored_messages = [self.system_message]

    def update_messages(self, message: BaseMessage) -> List[BaseMessage]:
        self.stored_messages.append(message)
        return self.stored_messages

    def step(
        self,
        input_message: HumanMessage,
    ) -> AIMessage:
        messages = self.update_messages(input_message)

        output_message = self.model(messages)
        self.update_messages(output_message)

        return output_message

In [ ]:
# Set up roles for role playing

assistant_role_name = "Market Researcher"
user_role_name = "UX Researcher"
task = "Given an idea for a new technology business, perform market research which includes: gauging market size, how to describe the target customer, what methods of research are suitable."
word_limit = 25 # word limit for task brainstorming

In the next step it would be ideal if the System Message could be the output of a compressed LLM output to hone in on the goal more quickly.

On further reading the CAMEL Framework is not able to be integrated with the other tools in the LangChain Library. The prompts are also carefully crafted, in order to maintain safety and cannot be changed.

In [ ]:
task_specifier_sys_msg = SystemMessage(content="Restrict Research to USA only")
task_specifier_prompt = (
"""Here is a task that {assistant_role_name} will help {user_role_name} to complete: {task}.
Please make it more specific. Be creative and imaginative.
Please reply with the specified task in {word_limit} words or less. Do not add anything else."""
)
task_specifier_template = HumanMessagePromptTemplate.from_template(template=task_specifier_prompt)
task_specify_agent = CAMELAgent(task_specifier_sys_msg, ChatOpenAI(temperature=1.1))
task_specifier_msg = task_specifier_template.format_messages(assistant_role_name=assistant_role_name,
                                                             user_role_name=user_role_name,
                                                             task=task, word_limit=word_limit)[0]
specified_task_msg = task_specify_agent.step(task_specifier_msg)
print(f"Specified task: {specified_task_msg.content}")
specified_task = specified_task_msg.content